In [94]:
import numpy as np
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from gensim.models import Word2Vec
from keras.models import Sequential, load_model
import keras.backend as K
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import cohen_kappa_score
from keras.layers import LSTM, Dropout, Dense

**Preparing Dataset**

In [95]:
# Load the dataset
df = pd.read_csv("Dataset/training_set_rel3.tsv", sep='\t', encoding='ISO-8859-1')
df.dropna(axis=1, inplace=True)
df.drop(columns=['domain1_score', 'rater1_domain1', 'rater2_domain1'], inplace=True, axis=1)


In [96]:
# Load processed data
temp = pd.read_csv("Processed_data.csv")
temp.drop("Unnamed: 0", inplace=True, axis=1)
df['domain1_score'] = temp['final_score']

In [97]:
# Split into features and target
y = df['domain1_score']
df.drop('domain1_score', inplace=True, axis=1)
X = df

In [98]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [99]:
#Make Dataset
y = df['domain1_score']
df.drop('domain1_score',inplace=True,axis=1)
X=df

KeyError: 'domain1_score'

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
X_train.shape

(9083, 3)

**PREPROCESSING**

In [100]:
train_e = X_train['essay'].tolist()
test_e = X_test['essay'].tolist()

In [101]:
# Define preprocessing functions
def sent2word(x):
    x = re.sub("[^A-Za-z]", " ", x)
    x.lower()
    filtered_sentence = [] 
    words = x.split()
    for w in words:
        if w not in stop_words: 
            filtered_sentence.append(w)
    return filtered_sentence

In [102]:
len(train_sents)

116500

In [103]:
train_sents[0]

['It',
 'first',
 'day',
 'high',
 'school',
 'gut',
 'full',
 'butterflies',
 'make',
 'want',
 'run',
 'bathrooms',
 'hide',
 'world']

**Preparing WORD2VEC and LSTM Model**

In [104]:
def get_model():
    model = Sequential()
    model.add(LSTM(300, dropout=0.4, recurrent_dropout=0.4, input_shape=[1, 300], return_sequences=True))
    model.add(LSTM(64, recurrent_dropout=0.4))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='relu'))
    model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['mae'])
    model.summary()
    return model

In [105]:
# Training Word2Vec model
num_features = 300
min_word_count = 40
num_workers = 4
context = 10
downsampling = 1e-3

model = Word2Vec(train_sents,
                 workers=num_workers,
                 vector_size=num_features,
                 min_count=min_word_count,
                 window=context,
                 sample=downsampling)

model.init_sims(replace=True)
model.wv.save_word2vec_format('word2vecmodel.bin', binary=True)

C:\Users\Yuvraj Singh\AppData\Local\Temp\ipykernel_11916\2073450766.py:15: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


In [106]:
def makeVec(words, model, num_features):
    vec = np.zeros((num_features,), dtype="float32")
    noOfWords = 0.
    index2word_set = set(model.wv.index_to_key)
    for i in words:
        if i in index2word_set:
            noOfWords += 1
            vec = np.add(vec, model.wv.get_vector(i))
    vec = np.divide(vec, noOfWords)
    return vec

def getVecs(essays, model, num_features):
    c=0
    essay_vecs = np.zeros((len(essays),num_features),dtype="float32")
    for i in essays:
        essay_vecs[c] = makeVec(i, model, num_features)
        c+=1
    return essay_vecs


clean_train=[]
for i in train_e:
    clean_train.append(sent2word(i))
training_vectors = getVecs(clean_train, model, num_features)

clean_test=[]

for i in test_e:
    clean_test.append(sent2word(i))
testing_vectors = getVecs(clean_test, model, num_features)

C:\Users\Yuvraj Singh\AppData\Local\Temp\ipykernel_11916\1079385726.py:9: RuntimeWarning: invalid value encountered in divide
  vec = np.divide(vec, noOfWords)


In [107]:
training_vectors.shape

(9083, 300)

In [108]:
training_vectors = np.array(training_vectors)
testing_vectors = np.array(testing_vectors)

# Reshaping train and test vectors to 3 dimensions. (1 represnts one timestep)
training_vectors = np.reshape(training_vectors, (training_vectors.shape[0], 1, training_vectors.shape[1]))
testing_vectors = np.reshape(testing_vectors, (testing_vectors.shape[0], 1, testing_vectors.shape[1]))
lstm_model = get_model()

C:\Users\Yuvraj Singh\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                   │ (None, 1, 300)         │       721,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 64)             │        93,440 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 814,705 (3.11 MB)

 Trainable params: 814,705 (3.11 MB)

 Non-trainable params: 0 (0.00 B)

In [109]:
training_vectors.shape

(9083, 1, 300)


**TRAINING AND PREDICTION**







In [110]:
lstm_model.fit(training_vectors, y_train, batch_size=64, epochs=200)

Epoch 1/200


142/142 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 20.2119 - mae: 3.6992
Epoch 2/200
142/142 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 5.6054 - mae: 1.8805
Epoch 3/200
142/142 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 5.3413 - mae: 1.8323
Epoch 4/200
142/142 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 5.1141 - mae: 1.7870
Epoch 5/200
142/142 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 5.0431 - mae: 1.7882
Epoch 6/200
142/142 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 4.9883 - mae: 1.7605
Epoch 7/200
142/142 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 5.0973 - mae: 1.7803
Epoch 8/200
142/142 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 5.1110 - mae: 1.7848
Epoch 9/200
142/142 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 5.1049 - mae: 1.7945
Epoch 10/200
142/142 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 5.0209 - mae: 1.7642
Epoch 11/200
142/142 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 5.0248 - mae: 1.7598
Epoch 12/200
142/142 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 5.0227 - mae: 1.7603
Epoch 13/200
142/142 ━━━

In [111]:
lstm_model.save('final_lstm.h5')
y_pred = lstm_model.predict(testing_vectors)
y_pred = np.around(y_pred)
y_pred

122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


array([[3.],
       [5.],
       [7.],
       ...,
       [8.],
       [7.],
       [9.]], dtype=float32)